In [27]:
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py

!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
!wget https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/threats.txt

import functions
from functions import *
import spacy
nlp = spacy.load('en_core_web_sm')#, disable=['parser', 'ner'])

!wget https://raw.githubusercontent.com/tylerneylon/explacy/master/explacy.py

--2021-04-11 22:30:43--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6913 (6.8K) [text/plain]
Saving to: ‘functions.py.2’

functions.py.2      100%[===================>]   6.75K  --.-KB/s    in 0s      

2021-04-11 22:30:43 (48.7 MB/s) - ‘functions.py.2’ saved [6913/6913]

--2021-04-11 22:30:43--  https://raw.githubusercontent.com/carlosmlosa/nlp_sentences_processing/main/files/assets.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 395

In [28]:
import explacy

sentence = 'Technical Complexity and Delayed Delivery threatens Processes with impact 2 and probability 3.'
             
nlp = spacy.load('en')

# explacy.print_parse_info(nlp, sentences)


explacy.print_parse_info(nlp, sentence)

Dep tree      Token       Dep type Lemma       Part of Sp
───────────── ─────────── ──────── ─────────── ──────────
          ┌─► Technical   compound Technical   PROPN     
    ┌─►┌──┼── Complexity  nsubj    Complexity  PROPN     
    │  │  └─► and         cc       and         CCONJ     
    │  │  ┌─► Delayed     compound Delayed     PROPN     
    │  └─►└── Delivery    conj     Delivery    PROPN     
┌┬──┴─────┬── threatens   ROOT     threaten    VERB      
││        └─► Processes   dobj     process     NOUN      
│└─►┌──────── with        prep     with        ADP       
│   └─►┌─┬┬── impact      pobj     impact      NOUN      
│      │ │└─► 2           nummod   2           NUM       
│      │ └──► and         cc       and         CCONJ     
│      └─►┌── probability conj     probability NOUN      
│         └─► 3           nummod   3           NUM       
└───────────► .           punct    .           PUNCT     


In [29]:
import functions
from functions import *

threats_file = '/content/threats.txt'
threats_dict = terms_dict(threats_file)




# Añadimos el termino 'intentional' a todos los términos que tengan la palabra 
# 'deliberated' porque 'deliberated' da problemas y sugerimos evitar esa palabra
for k,v in threats_dict.items():
  if 'Deliberated' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'intentional')

# añadimos las palabras para el acrónimo HW
for k,v in threats_dict.items():
  if 'HW' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'hardware')

# añadimos las palabras para el acrónimo SW
for k,v in threats_dict.items():
  if 'SW' in threats_dict[k]:
    add_word_to_dict(threats_dict,k, 'software')


assets_file = '/content/assets.txt'
assets_dict = terms_dict(assets_file)

In [30]:
def process_asset_threats_sentence(sentence: str, result: {}):
  """Function that process a sentences finding the terms:
  asset, threatType1, threatType2, prob and impact"""

  sentence = sentence.replace(",",".") # Para admitir decimales con "," o "."
  doc = nlp(str(sentence)) 
  
  asset = extract_dobject(sentence)      
  asset = clean(asset)
  asset_matches = search_matches(assets_dict, asset)
  asset_count = counter(asset_matches)
  asset = higher_frecuency_term(asset_count)
  result["asset"] = asset


  raw_threats = extract_subject(sentence)
  threats= []
  aux = []
  for element in raw_threats:
    if element == "," or element == "and":
      threats.append(aux)
      aux = []
    elif element == raw_threats[-1]:
      threats.append(aux)
       
    else:
      aux.append(element)


 
  cont = 0
  for threat in threats:
    cont += 1
    threat = clean(threat)
    threat_matches = search_matches(threats_dict, threat)
    threat_count = counter(threat_matches)
    threat = higher_frecuency_term(threat_count)
    result["threatType"+str(cont)] = threat

  # threat = clean(threat)
  # threat_matches = search_matches(threats_dict, threat)  
  # threat_count = counter(threat_matches)  
  # threat = higher_frecuency_term(threat_count)  
  # result["threatType"] = threat

  result["prob"] = extract_prob(sentence)
  result["impact"] = extract_impact(sentence)



In [31]:
result= {}
sentence = 'Technical Complexity and Delayed Delivery threatens Processes with impact 2 and probability 3.'


process_asset_threats_sentence(sentence, result)
result = {"result": [result]}
write_json("results.json", result) # escribir archivo con los resultados
result

{'result': [{'asset': 'Processes',
   'impact': 2,
   'prob': 3,
   'threatType1': 'TechnicalComplexity',
   'threatType2': 'DelayedDelivery'}]}